In [1]:
import pandas as pd
import numpy as np
import pickle
import time
import random
import itertools

In [2]:
%%time
graph_dict = pd.read_feather('data/graph.ftr', use_threads=True).set_index('source_item_id').target_item_ids
print(len(graph_dict))
graph_dict[9036]

38488233
CPU times: user 16.9 s, sys: 3.29 s, total: 20.2 s
Wall time: 20 s


array([ 1132636,       60,   367211,        5, 12750167, 16086586,
       16098713, 37251206,   131566,   169470,    38104,    31519,
         205375,   632404,  1366018,  1517792,  5442484,   689775,
          93728,  1332315,   678263,  3332215,  1905051,  4920135,
         111734,  1326886,  1906857,  2267418,  3739104, 17552952,
       41799198,       90,     1860,     1085,     1781,    12152,
         127885,    43035,   101333,    49258,    13298,     9301,
        1280677,  1700481,     3617,   613201,       30,      397,
            150,      188,      652,    28513,     9067,   131964,
           9056,    83364,    93996,  5460604])

In [3]:
%%time
sentences = pickle.load(open("data/dataset.p", "rb"))
print(len(sentences))
sentences[0]

120000
CPU times: user 1.87 s, sys: 292 ms, total: 2.16 s
Wall time: 2.14 s


([array([  386083,  1335064,  6852012, 12090795])],
 array(['military cross'], dtype=object),
 ('Military Cross', 488249, 1335064))

In [4]:
def Traverse(root, levels=1):
    if (type(graph_dict.get(root))==None):
        a =[]
        return a
    level = [np.unique(graph_dict.get(root))]
    #print (type(level))
    for i in range(1, levels):
        try:
            temp = pd.unique(np.concatenate([graph_dict.get(j, np.array([], dtype=int)) for j in level[i-1]]))
        except:
            return []
        level.append(np.setdiff1d(temp, np.concatenate(level)))
    for i in range(len(level)):
        level[i]=set(level[i])
    return level


In [5]:
%%time
a=Traverse(9036,10)
len(a)

CPU times: user 46.6 s, sys: 67.5 ms, total: 46.7 s
Wall time: 46.7 s


10

In [6]:
def find_shortest_path_length(src,dst,maxlev,Dict):
    if dst==-1 or src ==-1:
        return 0
    if (src,dst) in Dict:
        return Dict[(src,dst)]
    else:
        pathlength=0
        found=False
        reachend=False
        level_sets=Traverse(src,maxlev)
        if level_sets==[]:
            Dict[(src,dst)]=5
            return 5
        level=0
    
        while ((not found) and (not reachend)):
            if (level==maxlev):
                reachend=True
        
            elif dst in level_sets[level]:
                pathlength=level
                found =True
            else:
                level+=1
        if found == True:
            Dict[(src,dst)]=level+1
            return level+1
        else:
            Dict[(src,dst)]=5
            return 5
        

In [7]:
%%time
find_shortest_path_length(9036,5507494,10, {})

CPU times: user 48.5 s, sys: 42.2 ms, total: 48.6 s
Wall time: 48.6 s


10

In [8]:
find_shortest_path_length(9036,-1,10,{})

0

In [9]:
id_counts_df = pd.read_csv("data/id_counts.csv").set_index('item_id')
display(id_counts_df)
item_views_dict = id_counts_df.views.to_dict()
item_counts_dict = id_counts_df.counts.to_dict()
print(item_views_dict[6199])
print(item_counts_dict[6199])

def sort_ids(candidate_ids, mode, n=1):
    if mode == 'count':
        tie_break_list = [item_counts_dict.get(i, 0) for i in candidate_ids]
    elif mode == 'view':
        tie_break_list = [item_views_dict.get(i, 0) for i in candidate_ids]
        
    return candidate_ids[np.argsort(tie_break_list)[::-1]]

,page_id,title,views,counts
item_id,,,,
6199,12,Anarchism,31335,3540
38404,25,Autism,49693,2114
101038,39,Albedo,14573,2825
9659,290,A,25859,175
173,303,Alabama,52765,11125
...,...,...,...,...
76894635,62470350,Daming Zhu,16,0
76894633,62470423,Tony Dews,7,2
76896959,62470432,Samsung PL20,9,0


31335
3540


In [10]:
#pairwise for all => takes extremely long
def disambiguate(sample):
    Dict={}
    candidate_ids = np.array(sample[0][0])
    viable_neighbor_ids = [i for i in sample[0][1:] if len(i) > 0]
    true_id = sample[2][2]
                           
    if len(candidate_ids) == 0 or true_id not in candidate_ids:
        return None
    if len(candidate_ids) == 1:
        return candidate_ids[0]
    if len(viable_neighbor_ids) == 0:
        #return the most popular candidate id if there are no nieghbors
        return sort_ids(candidate_ids, 'count')[0]
                           
    candidate_lists = [candidate_ids] + viable_neighbor_ids
    #print(candidate_lists)
    
    combination_list = list(itertools.product(*candidate_lists))
    #print(len(combination_list))
    disambiguation = combination_list[0]
    least = np.inf
    for combinations in combination_list:
        #print("dict is currently:")
        #print(Dict)
        distance_sum = 0
        #generate a list of pairs of candidates to calculate pairwise distances
        #print("current combination:", combinations)
        pairs = list(itertools.combinations(combinations,2))
        #print(pairs)
        #print(len(pairs))
        for pair in pairs:
            #print("current pair:", pair)
            pair_dist = find_shortest_path_length(pair[0],pair[1],3,Dict)
            #print("pair dist", pair_dist)
            distance_sum += pair_dist
        if (distance_sum < least):
            least = distance_sum
            disambiguation = combinations
        #print(least)
        #print("end of current combination==========")
    return disambiguation[0]

In [11]:
%%time
disambiguate(sentences[1]), sentences[1][2]

CPU times: user 7.38 s, sys: 39.5 ms, total: 7.42 s
Wall time: 7.42 s


(25, ('Wales', 69894, 25))

In [12]:
def prune_sentence(sentence):
    sentence_length = len(sentence[0])
    total_disamb_count = sum([len(i) for i in sentence[0]])
    return (sentence_length > 10 or total_disamb_count > 100)
            
pruned_sentences = [i for i in sentences if not prune_sentence(i)]
len(pruned_sentences)

103182

In [13]:
import random
def get_samples(dataset, n, seed=None):
    random.seed(seed)
    return random.sample(dataset, n)

samples = get_samples(pruned_sentences, 10000, 0)
print(len(samples))
samples[:5]

10000


[([array([    1218,    55502,   229348,   478463,   489607,   914332,
           1411710,  1687937,  1687939,  1687955,  1923705,  2091539,
           2116978,  2237261,  2398662,  3708812,  3808056,  4993181,
           5665712,  6185054,  6185068,  6185071,  6185073,  6185076,
           6185078,  6185082,  6185194, 10540001, 12024541, 14903165,
          16255155, 16995361, 18463567, 19265218, 19265220, 20645251,
          21958268, 26529979, 27940563, 28865002, 28942207, 30526957,
          31570539, 31804795, 46718882, 55517162, 56696301, 65527529,
          65564651, 72667984, 76627393, 76627823, 76629209]),
   array([2657964]),
   array([   47740,   398931,  2781527, 12242988, 16009614, 17410811,
          37543155]),
   [],
   [],
   array([425281]),
   array([193163])],
  array(['jerusalem', "solomon's stables", 'muslim',
         'some 600 square yards', '500 square metres', 'the al-aqsa mosque',
         'the temple mount'], dtype=object),
  ('Jerusalem', 16043, 1218)),
 ([a

## Disambiguation

https://arxiv.org/abs/1802.01074  
http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.557.136  
https://www.kaggle.com/gabrielaltay/kdwd-subclass-path-ner

In [14]:
%%time
# single process
def disamb_sentences(sentences):
    st = time.time()
    cor_count=0
    for count, sentence in enumerate(sentences):
        if disambiguate(sentence) == sentence[2][2]:
            cor_count += 1
        print(f'{count/len(sentences)}% done\t{time.time()-st}')
    return cor_count/len(sentences)

#disamb_sentences(get_samples(pruned_sentences, 10, 0))

CPU times: user 0 ns, sys: 5 µs, total: 5 µs
Wall time: 7.87 µs


In [15]:
import multiprocessing as mp
n_cores = mp.cpu_count()
n_cores

16

In [16]:
%%time
def disamb_sentence_mp(sample):
    count, sentence = sample
    print(f'{count}/{num_samples} {time.time()-st}\tprocessing: {sentence[2]}')
    return disambiguate(sentence) == sentence[2][2]

num_samples = 1000
mp_samples = zip(np.arange(num_samples), get_samples(pruned_sentences, num_samples, 0))

st=time.time()
p = mp.Pool(n_cores-1)
results = p.map(disamb_sentence_mp, mp_samples)  
p.close()
p.join()

sum(results)/len(results)

0/1000	1.3756780624389648	processing: ('Jerusalem', 16043, 1218)
17/1000	1.376389503479004	processing: ('Indiana', 21883857, 1415)
34/1000	1.3771779537200928	processing: ('Ramanagara district', 12690228, 427679)
51/1000	1.3781564235687256	processing: ('Otto Skorzeny', 72249, 89383)
35/1000	1.3796043395996094	processing: ('stamps with forged cancels', 6890367, 914901)
52/1000	1.381058931350708	processing: ('North Korea', 21255, 423)
68/1000	1.3800711631774902	processing: ('Senator of the College of Justice', 1883713, 7450071)
36/1000	1.3817055225372314	processing: ('University of Dallas', 550344, 3551578)
53/1000	1.3824553489685059	processing: ('2003', 190182, 782572)
85/1000	1.3816328048706055	processing: ('India', 14533, 668)
37/1000	1.3830294609069824	processing: ('United States Military', 32212, 11211)
54/1000	1.3837766647338867	processing: ('Francesca Michielin', 34366511, 439571)
102/1000	1.3827481269836426	processing: ('India', 14533, 668)
38/1000	1.3842556476593018	processing: (

6/1000	12.918056726455688	processing: ('libretto', 46950, 131084)
26/1000	12.91783356666565	processing: ('Morris Davis', 3679452, 14596232)
7/1000	12.920561790466309	processing: ('suspension', 426955, 272870)
8/1000	12.923489570617676	processing: ('Utah Territory', 452226, 128026)
27/1000	12.924414873123169	processing: ('Milwaukee, Wisconsin', 53117, 37836)
9/1000	12.926417350769043	processing: ('Antje Kempe', 9911212, 443993)
10/1000	12.936917304992676	processing: ('Hundred of Mitchell', 52346783, 28428400)
11/1000	12.938914775848389	processing: ('concurrency', 928467, 1414548)
12/1000	12.96205735206604	processing: ('Lecompton Constitution', 447568, 2995078)
13/1000	12.963932037353516	processing: ('military science fiction', 306754, 904447)
14/1000	12.96786117553711	processing: ('Onogur', 1142195, 762453)
15/1000	12.971555709838867	processing: ('The Jungle Book', 200145, 189509)
224/1000	13.125893831253052	processing: ('Buddhism', 3267529, 748)
225/1000	13.1274893283844	processing: ('

250/1000	51.57368612289429	processing: ('Red Hook, Brooklyn', 536839, 1781122)
251/1000	51.57509803771973	processing: ('fast bowler', 678695, 45085)
252/1000	51.57636523246765	processing: ('Beaux Arts', 440475, 1268134)
253/1000	51.57751727104187	processing: ('series', 11392921, 5247729)
254/1000	51.57879614830017	processing: ('FASMBS', 11789322, 4745025)
340/1000	51.58075737953186	processing: ('Saltbox', 1631102, 5771190)
341/1000	51.588643074035645	processing: ('amenable', 657430, 1857002)
342/1000	51.59002614021301	processing: ('Jon Cleary', 2632856, 3183180)
343/1000	51.90003800392151	processing: ('melodic death metal', 633174, 253918)
344/1000	51.90133500099182	processing: ('cloud computing', 19541494, 483639)
107/1000	52.155248165130615	processing: ('Olympiacos', 1229550, 19628)
108/1000	52.15678000450134	processing: ('Austria', 26964606, 40)
345/1000	53.17957282066345	processing: ('Valur', 1910601, 58414784)
346/1000	53.181180000305176	processing: ('Perpignan', 38707, 6730)
347/

231/1000	91.81510376930237	processing: ('Chancery Division', 26569402, 1617747)
232/1000	91.81653475761414	processing: ('Iranian Air Force', 412792, 378442)
233/1000	91.81851649284363	processing: ('Bundesliga', 686142, 82595)
234/1000	92.07571959495544	processing: ('Sydney, Australia', 27862, 3130)
235/1000	92.07730197906494	processing: ("Queen Elizabeth's Grammar School, Crediton", 19840752, 7270297)
236/1000	92.07931637763977	processing: ('South Shields', 368433, 279000)
339/1000	95.98964500427246	processing: ('urban fantasy', 607524, 1188977)
442/1000	95.99330806732178	processing: ('Campbeltown Loch', 1092825, 1030654)
443/1000	95.99706816673279	processing: ('tRNA', 532175, 201448)
444/1000	95.99977946281433	processing: ('arm', 53238, 43471)
445/1000	96.00159478187561	processing: ('film', 21555729, 11424)
237/1000	100.53644633293152	processing: ('All-American', 735314, 762853)
459/1000	100.53990721702576	processing: ('Komatsu Tatewaki', 18654001, 703847)
460/1000	100.54141521453857	

398/1000	135.94357872009277	processing: ('Holmavatnet', 4946897, 3888951)
399/1000	135.9466049671173	processing: ('cabinetmaker', 37962744, 16868432)
400/1000	135.9478793144226	processing: ('products', 540448, 542929)
499/1000	137.07175397872925	processing: ('model organism', 19374, 213907)
500/1000	137.07573556900024	processing: ('Allergic reactions', 55313, 42982)
501/1000	137.07765936851501	processing: ('Georgia State Senate', 964172, 65122131)
502/1000	137.07963013648987	processing: ('Nero', 21632, 1413)
90/1000	139.270574092865	processing: ('outfielder', 528703, 1142885)
91/1000	139.27209997177124	processing: ('sandbar', 295148, 28337)
92/1000	139.49391055107117	processing: ('University of Auckland', 277456, 492467)
93/1000	139.4954240322113	processing: ('eun', 21290463, 17055707)
535/1000	139.6318383216858	processing: ('World War II', 32927, 362)
536/1000	140.80650782585144	processing: ('Northern Rock', 499858, 1130757)
537/1000	140.80840063095093	processing: ('1991 NatWest Troph

489/1000	198.4710099697113	processing: ('Aza', 13158946, 1113505)
490/1000	198.47318053245544	processing: ('First Nations communities in Canada', 196075, 863284)
491/1000	198.47516894340515	processing: ('Kuala Lumpur', 16854, 1865)
665/1000	198.97982239723206	processing: ('poet', 25353172, 49757)
492/1000	199.71604776382446	processing: ('Nanchang', 386882, 171943)
648/1000	200.90846610069275	processing: ('Templeport', 27743762, 2974557)
666/1000	202.7403528690338	processing: ('Brazil', 3383, 155)
579/1000	208.20912313461304	processing: ('Orson Welles', 22196, 24829)
649/1000	209.34135460853577	processing: ('Able', 30461907, 272306)
650/1000	209.34435415267944	processing: ('Jekyll', 343872, 452354)
580/1000	209.53843545913696	processing: ('Dylan Lewis', 5502039, 5318739)
581/1000	209.54276156425476	processing: ('Radio & Records', 2018789, 7280587)
582/1000	209.55566382408142	processing: ('puck', 53347, 508918)
583/1000	209.55888557434082	processing: ('PlayStation VR', 42365665, 16011703

557/1000	246.62121772766113	processing: ('George Frideric Handel', 12775, 7302)
558/1000	248.15630269050598	processing: ('Cook Islands', 7067, 26988)
720/1000	249.76352882385254	processing: ('reclamation', 24536365, 13642695)
721/1000	249.7651572227478	processing: ('fossils', 10958, 40614)
722/1000	249.7671332359314	processing: ('Stanislav Zvolenský', 15848657, 1382674)
723/1000	249.78334879875183	processing: ('University of Manchester', 83250, 867530)
724/1000	249.78467917442322	processing: ('1928 Winter Olympics', 49369, 9591)
725/1000	249.78581523895264	processing: ('Liberals', 483222, 3366499)
726/1000	249.78701305389404	processing: ('Dreamcast', 29036, 184198)
751/1000	250.43280839920044	processing: ('Priyamani', 9272179, 3519936)
752/1000	250.43462204933167	processing: ('United States Coast Guard', 32223, 11224)
753/1000	250.4359483718872	processing: ('Islamic State', 9087364, 2429253)
638/1000	252.13479375839233	processing: ('Chief Executive Officer', 52234, 484876)
639/1000	252

832/1000	282.8484992980957	processing: ('LeBron James', 240940, 36159)
901/1000	282.851478099823	processing: ('California Secretary of State', 1074529, 7024182)
902/1000	282.8529372215271	processing: ("children's novel", 52847, 131539)
903/1000	282.85475611686707	processing: ('North Carolina General Assembly', 426283, 10566263)
904/1000	282.85643792152405	processing: ('Rick Rock', 6800719, 7331688)
905/1000	282.863840341568	processing: ('Hindi', 13652, 1568)
886/1000	283.6467754840851	processing: ('Xizhimen', 951878, 4567887)
887/1000	283.6484341621399	processing: ('Jiske Griffioen', 33744508, 1409435)
888/1000	283.64982175827026	processing: ('Macha Oromo', 37235669, 1294158)
889/1000	283.65214586257935	processing: ('Cuddapah', 3505837, 30651)
890/1000	283.6549828052521	processing: ("men's coxless pair", 45685219, 22968985)
891/1000	283.6569359302521	processing: ('FIFA 08', 11108427, 842259)
892/1000	283.6586892604828	processing: ('Byzantine', 16972981, 12544)
893/1000	283.660596609115

812/1000	318.58360743522644	processing: ('footballer', 10568, 2736)
813/1000	318.5865046977997	processing: ('Natwar Singh', 748850, 1285090)
814/1000	318.58803939819336	processing: ('video game music', 350752, 1062702)
815/1000	318.5937428474426	processing: ('Schneider Trophy', 56291, 959841)
900/1000	320.1810338497162	processing: ('books and toys', 697240, 740777)
988/1000	320.87811374664307	processing: ('Madhya Pradesh', 47945, 1188)
878/1000	324.051057100296	processing: ('Euro 2016', 7932564, 189571)
879/1000	324.0523314476013	processing: ('Sri Lanka', 26750, 854)
928/1000	330.7342965602875	processing: ('Isles of Scilly Steamship Company', 3498941, 6084234)
929/1000	330.73588395118713	processing: ('Scotland', 26994, 22)
738/1000	333.7411184310913	processing: ('Karnataka', 16880, 1185)
989/1000	335.3192913532257	processing: ('defender', 649702, 336286)
739/1000	339.68696570396423	processing: ('Sony PlayStation', 13392949, 1323662)
740/1000	339.6882965564728	processing: ('New Zealand'

0.651